## Read and Clean Dataset


Import the libraries.


In [1]:
# Install tensorflow and keras in Anaconda
# !pip install tensorflow
# !pip install keras

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import metrics

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Read the data into a <em>pandas</em> dataframe.


In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


#### Let's check how many data points we have.


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors (X) and target (y)


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

X = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
y = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [8]:
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [10]:
X_norm = (X - X.mean()) / X.std()
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [11]:
n_cols = X_norm.shape[1] # number of predictors
print("Number of features:", n_cols)

Number of features: 8


<a id="item1"></a>


<a id='item32'></a>


## Build a Neural Network


Let's define a function that defines our neural network model for us so that we can conveniently call it to create our model.


In [12]:
# define regression model
def create_neural_network():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layers, each of 10 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [13]:
# build the model
model = create_neural_network()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [14]:
# fit the model
mean_sq_error = [] # Empty list of mean squared error

def split_and_fit_model(myModel):
    X_train, X_test, y_train, y_test = train_test_split( X_norm, y, test_size=0.3, random_state=4)
    
    myModel.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=2)
    
    # evaluate the model
    scores = myModel.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: {} \n Error: {}".format(scores, 100-scores*100))
    
    # Get the predicted values
    yhat = myModel.predict(X_test)
    mse_sk = metrics.mean_squared_error(y_test, yhat) # Calculate RMSE
    mean_sq_error.append(mse_sk)

# Step 4
for i in range(50):
    print("Iteration %s:\n" %str(i))
    split_and_fit_model(model)
    
print(len(mean_sq_error))
print("Mean:", np.mean(np.array(mean_sq_error)))
print("Standard Deviation :", np.std(np.array(mean_sq_error)))

Iteration 0:

Epoch 1/100
16/16 - 1s - loss: 1603.8373 - val_loss: 1488.3717 - 571ms/epoch - 36ms/step
Epoch 2/100
16/16 - 0s - loss: 1591.8435 - val_loss: 1477.0277 - 37ms/epoch - 2ms/step
Epoch 3/100
16/16 - 0s - loss: 1580.1207 - val_loss: 1465.6403 - 37ms/epoch - 2ms/step
Epoch 4/100
16/16 - 0s - loss: 1568.2263 - val_loss: 1454.3510 - 39ms/epoch - 2ms/step
Epoch 5/100
16/16 - 0s - loss: 1556.3639 - val_loss: 1442.8470 - 39ms/epoch - 2ms/step
Epoch 6/100
16/16 - 0s - loss: 1544.6816 - val_loss: 1430.8751 - 39ms/epoch - 2ms/step
Epoch 7/100
16/16 - 0s - loss: 1532.2531 - val_loss: 1419.2003 - 35ms/epoch - 2ms/step
Epoch 8/100
16/16 - 0s - loss: 1520.1205 - val_loss: 1406.9979 - 37ms/epoch - 2ms/step
Epoch 9/100
16/16 - 0s - loss: 1507.5381 - val_loss: 1394.7136 - 44ms/epoch - 3ms/step
Epoch 10/100
16/16 - 0s - loss: 1494.8850 - val_loss: 1381.8912 - 41ms/epoch - 3ms/step
Epoch 11/100
16/16 - 0s - loss: 1481.5515 - val_loss: 1369.1848 - 37ms/epoch - 2ms/step
Epoch 12/100
16/16 - 0s -

Epoch 96/100
16/16 - 0s - loss: 247.6875 - val_loss: 220.5049 - 39ms/epoch - 2ms/step
Epoch 97/100
16/16 - 0s - loss: 243.7689 - val_loss: 217.3269 - 36ms/epoch - 2ms/step
Epoch 98/100
16/16 - 0s - loss: 240.0397 - val_loss: 214.4170 - 69ms/epoch - 4ms/step
Epoch 99/100
16/16 - 0s - loss: 236.5126 - val_loss: 211.5191 - 54ms/epoch - 3ms/step
Epoch 100/100
16/16 - 0s - loss: 233.1569 - val_loss: 208.8367 - 42ms/epoch - 3ms/step
Accuracy: 237.6024932861328 
 Error: -23660.24932861328
10/10 [==============================] - 0s 1ms/step
Iteration 1:

Epoch 1/100
16/16 - 0s - loss: 229.8588 - val_loss: 206.3179 - 71ms/epoch - 4ms/step
Epoch 2/100
16/16 - 0s - loss: 226.7818 - val_loss: 203.9727 - 40ms/epoch - 2ms/step
Epoch 3/100
16/16 - 0s - loss: 223.9403 - val_loss: 201.7090 - 42ms/epoch - 3ms/step
Epoch 4/100
16/16 - 0s - loss: 221.1924 - val_loss: 199.6143 - 38ms/epoch - 2ms/step
Epoch 5/100
16/16 - 0s - loss: 218.5190 - val_loss: 197.5900 - 38ms/epoch - 2ms/step
Epoch 6/100
16/16 - 0

Epoch 90/100
16/16 - 0s - loss: 120.5547 - val_loss: 124.9842 - 37ms/epoch - 2ms/step
Epoch 91/100
16/16 - 0s - loss: 119.8618 - val_loss: 124.3934 - 36ms/epoch - 2ms/step
Epoch 92/100
16/16 - 0s - loss: 119.2424 - val_loss: 123.8733 - 34ms/epoch - 2ms/step
Epoch 93/100
16/16 - 0s - loss: 118.6006 - val_loss: 123.3692 - 35ms/epoch - 2ms/step
Epoch 94/100
16/16 - 0s - loss: 117.9862 - val_loss: 122.8203 - 39ms/epoch - 2ms/step
Epoch 95/100
16/16 - 0s - loss: 117.3460 - val_loss: 122.2828 - 37ms/epoch - 2ms/step
Epoch 96/100
16/16 - 0s - loss: 116.7643 - val_loss: 121.7004 - 38ms/epoch - 2ms/step
Epoch 97/100
16/16 - 0s - loss: 116.1990 - val_loss: 121.1616 - 41ms/epoch - 3ms/step
Epoch 98/100
16/16 - 0s - loss: 115.5545 - val_loss: 120.6458 - 44ms/epoch - 3ms/step
Epoch 99/100
16/16 - 0s - loss: 114.9933 - val_loss: 120.1009 - 43ms/epoch - 3ms/step
Epoch 100/100
16/16 - 0s - loss: 114.4551 - val_loss: 119.6102 - 43ms/epoch - 3ms/step
Accuracy: 131.6879119873047 
 Error: -13068.791198730

Epoch 85/100
16/16 - 0s - loss: 85.0698 - val_loss: 94.9640 - 37ms/epoch - 2ms/step
Epoch 86/100
16/16 - 0s - loss: 84.8839 - val_loss: 94.8384 - 36ms/epoch - 2ms/step
Epoch 87/100
16/16 - 0s - loss: 84.7267 - val_loss: 94.6806 - 36ms/epoch - 2ms/step
Epoch 88/100
16/16 - 0s - loss: 84.5368 - val_loss: 94.5567 - 36ms/epoch - 2ms/step
Epoch 89/100
16/16 - 0s - loss: 84.3993 - val_loss: 94.3934 - 36ms/epoch - 2ms/step
Epoch 90/100
16/16 - 0s - loss: 84.1846 - val_loss: 94.2514 - 37ms/epoch - 2ms/step
Epoch 91/100
16/16 - 0s - loss: 84.0179 - val_loss: 94.2325 - 50ms/epoch - 3ms/step
Epoch 92/100
16/16 - 0s - loss: 83.8383 - val_loss: 94.1219 - 38ms/epoch - 2ms/step
Epoch 93/100
16/16 - 0s - loss: 83.6617 - val_loss: 93.9848 - 69ms/epoch - 4ms/step
Epoch 94/100
16/16 - 0s - loss: 83.4987 - val_loss: 93.9009 - 38ms/epoch - 2ms/step
Epoch 95/100
16/16 - 0s - loss: 83.4245 - val_loss: 93.8168 - 39ms/epoch - 2ms/step
Epoch 96/100
16/16 - 0s - loss: 83.1901 - val_loss: 93.6814 - 36ms/epoch - 2

16/16 - 0s - loss: 68.8386 - val_loss: 81.7894 - 34ms/epoch - 2ms/step
Epoch 82/100
16/16 - 0s - loss: 68.6328 - val_loss: 81.5647 - 36ms/epoch - 2ms/step
Epoch 83/100
16/16 - 0s - loss: 68.4868 - val_loss: 81.3666 - 35ms/epoch - 2ms/step
Epoch 84/100
16/16 - 0s - loss: 68.3597 - val_loss: 81.2151 - 35ms/epoch - 2ms/step
Epoch 85/100
16/16 - 0s - loss: 68.2222 - val_loss: 81.0063 - 32ms/epoch - 2ms/step
Epoch 86/100
16/16 - 0s - loss: 68.0895 - val_loss: 80.8763 - 36ms/epoch - 2ms/step
Epoch 87/100
16/16 - 0s - loss: 67.9042 - val_loss: 80.6770 - 35ms/epoch - 2ms/step
Epoch 88/100
16/16 - 0s - loss: 67.7525 - val_loss: 80.5377 - 36ms/epoch - 2ms/step
Epoch 89/100
16/16 - 0s - loss: 67.5220 - val_loss: 80.3539 - 36ms/epoch - 2ms/step
Epoch 90/100
16/16 - 0s - loss: 67.3840 - val_loss: 80.1339 - 37ms/epoch - 2ms/step
Epoch 91/100
16/16 - 0s - loss: 67.1878 - val_loss: 79.9103 - 36ms/epoch - 2ms/step
Epoch 92/100
16/16 - 0s - loss: 67.0180 - val_loss: 79.8504 - 37ms/epoch - 2ms/step
Epoch

...

16/16 - 0s - loss: 26.3964 - val_loss: 39.9813 - 34ms/epoch - 2ms/step
Epoch 33/100
16/16 - 0s - loss: 26.4127 - val_loss: 40.0612 - 31ms/epoch - 2ms/step
Epoch 34/100
16/16 - 0s - loss: 26.4170 - val_loss: 40.2545 - 35ms/epoch - 2ms/step
Epoch 35/100
16/16 - 0s - loss: 26.3785 - val_loss: 40.0851 - 37ms/epoch - 2ms/step
Epoch 36/100
16/16 - 0s - loss: 26.4264 - val_loss: 40.0484 - 34ms/epoch - 2ms/step
Epoch 37/100
16/16 - 0s - loss: 26.3860 - val_loss: 40.1089 - 37ms/epoch - 2ms/step
Epoch 38/100
16/16 - 0s - loss: 26.3612 - val_loss: 40.1719 - 37ms/epoch - 2ms/step
Epoch 39/100
16/16 - 0s - loss: 26.3940 - val_loss: 40.0680 - 36ms/epoch - 2ms/step
Epoch 40/100
16/16 - 0s - loss: 26.3606 - val_loss: 40.1228 - 37ms/epoch - 2ms/step
Epoch 41/100
16/16 - 0s - loss: 26.4159 - val_loss: 40.2507 - 37ms/epoch - 2ms/step
Epoch 42/100
16/16 - 0s - loss: 26.4108 - val_loss: 40.0081 - 36ms/epoch - 2ms/step
Epoch 43/100
16/16 - 0s - loss: 26.3976 - val_loss: 40.1099 - 35ms/epoch - 2ms/step
Epoch

Epoch 29/100
16/16 - 0s - loss: 26.2885 - val_loss: 40.1109 - 36ms/epoch - 2ms/step
Epoch 30/100
16/16 - 0s - loss: 26.2571 - val_loss: 40.0946 - 34ms/epoch - 2ms/step
Epoch 31/100
16/16 - 0s - loss: 26.3234 - val_loss: 40.0646 - 36ms/epoch - 2ms/step
Epoch 32/100
16/16 - 0s - loss: 26.2988 - val_loss: 39.9810 - 35ms/epoch - 2ms/step
Epoch 33/100
16/16 - 0s - loss: 26.2662 - val_loss: 40.0695 - 36ms/epoch - 2ms/step
Epoch 34/100
16/16 - 0s - loss: 26.2944 - val_loss: 40.1536 - 35ms/epoch - 2ms/step
Epoch 35/100
16/16 - 0s - loss: 26.3185 - val_loss: 40.0045 - 35ms/epoch - 2ms/step
Epoch 36/100
16/16 - 0s - loss: 26.2763 - val_loss: 40.1396 - 37ms/epoch - 2ms/step
Epoch 37/100
16/16 - 0s - loss: 26.3304 - val_loss: 39.9297 - 39ms/epoch - 2ms/step
Epoch 38/100
16/16 - 0s - loss: 26.2481 - val_loss: 40.1365 - 38ms/epoch - 2ms/step
Epoch 39/100
16/16 - 0s - loss: 26.2848 - val_loss: 40.1029 - 37ms/epoch - 2ms/step
Epoch 40/100
16/16 - 0s - loss: 26.2799 - val_loss: 40.0933 - 36ms/epoch - 2

16/16 - 0s - loss: 25.9471 - val_loss: 39.4918 - 34ms/epoch - 2ms/step
Epoch 26/100
16/16 - 0s - loss: 25.9166 - val_loss: 39.4371 - 35ms/epoch - 2ms/step
Epoch 27/100
16/16 - 0s - loss: 25.9171 - val_loss: 39.5722 - 36ms/epoch - 2ms/step
Epoch 28/100
16/16 - 0s - loss: 25.9295 - val_loss: 39.5372 - 36ms/epoch - 2ms/step
Epoch 29/100
16/16 - 0s - loss: 25.9157 - val_loss: 39.5511 - 36ms/epoch - 2ms/step
Epoch 30/100
16/16 - 0s - loss: 25.9201 - val_loss: 39.3939 - 36ms/epoch - 2ms/step
Epoch 31/100
16/16 - 0s - loss: 25.9066 - val_loss: 39.4977 - 36ms/epoch - 2ms/step
Epoch 32/100
16/16 - 0s - loss: 25.8709 - val_loss: 39.4646 - 37ms/epoch - 2ms/step
Epoch 33/100
16/16 - 0s - loss: 25.9036 - val_loss: 39.4544 - 36ms/epoch - 2ms/step
Epoch 34/100
16/16 - 0s - loss: 25.9141 - val_loss: 39.5148 - 65ms/epoch - 4ms/step
Epoch 35/100
16/16 - 0s - loss: 25.9015 - val_loss: 39.3661 - 34ms/epoch - 2ms/step
Epoch 36/100
16/16 - 0s - loss: 25.8815 - val_loss: 39.3601 - 36ms/epoch - 2ms/step
Epoch

Epoch 22/100
16/16 - 0s - loss: 25.6667 - val_loss: 39.3978 - 34ms/epoch - 2ms/step
Epoch 23/100
16/16 - 0s - loss: 25.6250 - val_loss: 39.4011 - 34ms/epoch - 2ms/step
Epoch 24/100
16/16 - 0s - loss: 25.6116 - val_loss: 39.3820 - 36ms/epoch - 2ms/step
Epoch 25/100
16/16 - 0s - loss: 25.6290 - val_loss: 39.4223 - 35ms/epoch - 2ms/step
Epoch 26/100
16/16 - 0s - loss: 25.6641 - val_loss: 39.6314 - 33ms/epoch - 2ms/step
Epoch 27/100
16/16 - 0s - loss: 25.6103 - val_loss: 39.3724 - 35ms/epoch - 2ms/step
Epoch 28/100
16/16 - 0s - loss: 25.6096 - val_loss: 39.3476 - 36ms/epoch - 2ms/step
Epoch 29/100
16/16 - 0s - loss: 25.5970 - val_loss: 39.3268 - 35ms/epoch - 2ms/step
Epoch 30/100
16/16 - 0s - loss: 25.6442 - val_loss: 39.4179 - 36ms/epoch - 2ms/step
Epoch 31/100
16/16 - 0s - loss: 25.6600 - val_loss: 39.3133 - 36ms/epoch - 2ms/step
Epoch 32/100
16/16 - 0s - loss: 25.6011 - val_loss: 39.3545 - 37ms/epoch - 2ms/step
Epoch 33/100
16/16 - 0s - loss: 25.6172 - val_loss: 39.3863 - 37ms/epoch - 2

16/16 - 0s - loss: 25.5259 - val_loss: 39.6257 - 33ms/epoch - 2ms/step
Epoch 19/100
16/16 - 0s - loss: 25.4617 - val_loss: 39.4816 - 33ms/epoch - 2ms/step
Epoch 20/100
16/16 - 0s - loss: 25.5078 - val_loss: 39.1780 - 35ms/epoch - 2ms/step
Epoch 21/100
16/16 - 0s - loss: 25.4603 - val_loss: 39.3560 - 34ms/epoch - 2ms/step
Epoch 22/100
16/16 - 0s - loss: 25.4635 - val_loss: 39.3175 - 35ms/epoch - 2ms/step
Epoch 23/100
16/16 - 0s - loss: 25.5228 - val_loss: 39.5358 - 48ms/epoch - 3ms/step
Epoch 24/100
16/16 - 0s - loss: 25.4967 - val_loss: 39.3433 - 35ms/epoch - 2ms/step
Epoch 25/100
16/16 - 0s - loss: 25.4467 - val_loss: 39.2209 - 36ms/epoch - 2ms/step
Epoch 26/100
16/16 - 0s - loss: 25.4746 - val_loss: 39.3287 - 36ms/epoch - 2ms/step
Epoch 27/100
16/16 - 0s - loss: 25.4641 - val_loss: 39.3692 - 34ms/epoch - 2ms/step
Epoch 28/100
16/16 - 0s - loss: 25.4675 - val_loss: 39.4221 - 34ms/epoch - 2ms/step
Epoch 29/100
16/16 - 0s - loss: 25.4576 - val_loss: 39.4343 - 34ms/epoch - 2ms/step
Epoch

<a id="item4"></a>

<a id="item4"></a>

The mean from part B (with 50 epochs): 63.55776134625607

The mean from part C (with 100 epochs): 46.65215624259431

The mean in step C is less than in B